# Reducing the number of vertices/faces in a 3D model :

In [1]:
# Import Libraries
import numpy as np
import time
from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.GLU import *

## 1. Decode an .obj file

In [2]:
class ObjModel:
    def __init__(self, vertices=None, faces=None, normals=None, textures=None):
        self.vertices = vertices or []  # List of tuples (x, y, z)
        self.faces = faces or []  # List of faces (vertex indices)
        self.normals = normals or []  # List of normals
        self.textures = textures or []  # List of texture coordinates

    def load(self, filename):
        with open(filename, 'r') as file:
            for line in file:
                parts = line.split()
                if not parts:
                    continue
                if parts[0] == 'v':
                    self.vertices.append(tuple(map(float, parts[1:4])))
                elif parts[0] == 'f':
                    self.faces.append([int(i.split('/')[0]) - 1 for i in parts[1:]])
                elif parts[0] == 'vn':
                    self.normals.append(tuple(map(float, parts[1:4])))
                elif parts[0] == 'vt':
                    self.textures.append(tuple(map(float, parts[1:3])))

    def __str__(self):
        return f"Vertices: {len(self.vertices)}, Faces: {len(self.faces)}"

## 2. Implementing Mesh Reduction Algorithms :
### a. Vertex Clustering

In [3]:
def vertex_clustering(model, grid_size):
    min_vals = np.min(model.vertices, axis=0)
    grid = {}
    index_map = {}

    for i, vertex in enumerate(model.vertices):
        grid_pos = tuple(np.floor((np.array(vertex) - min_vals) / grid_size).astype(int))
        if grid_pos not in grid:
            grid[grid_pos] = []
        grid[grid_pos].append(i)

    new_vertices = []
    for indices in grid.values():
        centroid = np.mean([model.vertices[i] for i in indices], axis=0)
        new_index = len(new_vertices)
        new_vertices.append(centroid)
        for idx in indices:
            index_map[idx] = new_index

    new_faces = []
    for face in model.faces:
        new_face = [index_map[v] for v in face]
        if len(set(new_face)) == len(new_face):
            new_faces.append(new_face)

    return ObjModel(new_vertices, new_faces, model.normals, model.textures)

### b. Edge Collapse

In [4]:
def edge_collapse(model, threshold):
    vertices = np.array(model.vertices)
    edges = {}

    for face in model.faces:
        for i in range(len(face)):
            v1, v2 = sorted((face[i], face[(i + 1) % len(face)]))
            edges[(v1, v2)] = np.linalg.norm(vertices[v1] - vertices[v2])

    sorted_edges = sorted(edges.items(), key=lambda x: x[1])

    collapsed = set()
    for (v1, v2), length in sorted_edges:
        if length > threshold:
            break
        if v1 in collapsed or v2 in collapsed:
            continue
        vertices[v1] = (vertices[v1] + vertices[v2]) / 2
        collapsed.add(v2)

    new_faces = []
    for face in model.faces:
        new_face = [v for v in face if v not in collapsed]
        if len(new_face) == 3:
            new_faces.append(new_face)

    return ObjModel(vertices.tolist(), new_faces, model.normals, model.textures)

## 3. Save the resulting object in a new .obj file

In [5]:
def save_obj(model, filename):
    with open(filename, 'w') as file:
        for vertex in model.vertices:
            file.write(f"v {' '.join(map(str, vertex))}\n")
        for face in model.faces:
            file.write(f"f {' '.join(str(idx + 1) for idx in face)}\n")

In [6]:
# Directory and processing
for i in range(1, 7):
    filename = f"DVI2/obj{i}.obj"
    model = ObjModel()
    model.load(filename)

    grid_size = 0.1
    threshold = 0.1

    # Original model
    print(f"/////////// OBJ{i} ////////////")
    print("--- Original Model ---")
    print(str(model))

    # Vertex Clustering
    start_time = time.time()
    vc_model = vertex_clustering(model, grid_size)
    vc_time = time.time() - start_time
    save_obj(vc_model, f"DVI1/obj{i}_vertex_clustering.obj")
    print("--- Vertex Clustering ---")
    print(f"{str(vc_model)} | Time: {vc_time:.2f}s")

    # Edge Collapse
    start_time = time.time()
    ec_model = edge_collapse(model, threshold)
    ec_time = time.time() - start_time
    save_obj(ec_model, f"DVI1/obj{i}_edge_collapse.obj")
    print("--- Edge Collapse ---")
    print(f"{str(ec_model)} | Time: {ec_time:.2f}s\n")

/////////// OBJ1 ////////////
--- Original Model ---
Vertices: 1369, Faces: 2734
--- Vertex Clustering ---
Vertices: 1259, Faces: 2514 | Time: 0.03s
--- Edge Collapse ---
Vertices: 1369, Faces: 2102 | Time: 0.08s

/////////// OBJ2 ////////////
--- Original Model ---
Vertices: 10505, Faces: 20822
--- Vertex Clustering ---
Vertices: 752, Faces: 1613 | Time: 0.09s
--- Edge Collapse ---
Vertices: 10505, Faces: 0 | Time: 0.37s

/////////// OBJ3 ////////////
--- Original Model ---
Vertices: 2165, Faces: 4438
--- Vertex Clustering ---
Vertices: 1452, Faces: 2910 | Time: 0.04s
--- Edge Collapse ---
Vertices: 2165, Faces: 1574 | Time: 0.07s

/////////// OBJ4 ////////////
--- Original Model ---
Vertices: 17403, Faces: 34643
--- Vertex Clustering ---
Vertices: 451, Faces: 988 | Time: 0.14s
--- Edge Collapse ---
Vertices: 17403, Faces: 0 | Time: 0.58s

/////////// OBJ5 ////////////
--- Original Model ---
Vertices: 1932, Faces: 3824
--- Vertex Clustering ---
Vertices: 1932, Faces: 3824 | Time: 0.04